In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
import seaborn as sn
import numpy as np
from numpy.random import seed
import matplotlib
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

2024-03-21 14:15:55.742676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 14:15:55.742733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 14:15:55.787528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 14:15:55.892749: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 14:15:57.026913: W tensorflow/compiler/tf2

In [2]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

seaborn    : 0.13.2
numpy      : 1.23.5
torch      : 2.2.1
torchvision: 0.16.2
matplotlib : 3.5.2



In [3]:
# Verifica a disponibilidade do uso da GPU (RTX 3060 Ti) para o treinamento do modelo
!nvidia-smi
if torch.cuda.is_available():
    print(f"GPU atual: {torch.cuda.get_device_name(0)}")
else:
    print("Sem GPU disponível!")

Thu Mar 21 14:15:59 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.65                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:09:00.0  On |                  N/A |
| 52%   34C    P8             19W /  225W |     564MiB /   8192MiB |     14%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
torch.manual_seed(8499)
seed(4899)
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation(10),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                               transforms.Resize([200,200])]) #128,128

In [5]:
# Dados de Treino
train_data = ImageFolder(root = '/home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Training', 
                         transform=transform)
print(train_data)

Dataset ImageFolder
    Number of datapoints: 5712
    Root location: /home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Training
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
               Resize(size=[200, 200], interpolation=bilinear, max_size=None, antialias=True)
           )


In [6]:
# Dados de Teste
test_data = ImageFolder(root = '/home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Testing',
                       transform=transform)
print(test_data)

Dataset ImageFolder
    Number of datapoints: 1311
    Root location: /home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Testing
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
               Resize(size=[200, 200], interpolation=bilinear, max_size=None, antialias=True)
           )


In [7]:
# Número de amostras de treino
num_amostras_treino = len(train_data)
num_amostras_treino

5712

In [8]:
# Criamos um índice e o tornamos randômico
torch.manual_seed(8499)
seed(4899)
indices = list(range(num_amostras_treino))
np.random.shuffle(indices)

In [9]:
# Percentual dos dados de treino que usaremos no dataset de validação
valid_size = 0.25

In [10]:
# Agora fazemos o split para os dados de treino e validação
split = int(np.floor(valid_size * num_amostras_treino))
idx_treino, idx_valid = indices[split:], indices[:split]

In [11]:
# Definimos as amostras de treino
amostras_treino = SubsetRandomSampler(idx_treino)

In [12]:
# Definimos as amostras de validação
amostras_valid = SubsetRandomSampler(idx_valid)

In [13]:
# Número de subprocessos para carregar os dados
num_workers = 0
batch_size = 23

In [14]:
# Train Loader
torch.manual_seed(8499)
seed(4899)
train_loader = torch.utils.data.DataLoader(train_data, 
                                            batch_size = 28, 
                                            sampler = amostras_treino, 
                                            num_workers = num_workers)

In [15]:
# Data Loader de dados de validação
torch.manual_seed(8499)
seed(4899)
valid_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size = 28, 
                                           sampler = amostras_valid, 
                                           num_workers = num_workers)

In [16]:
# Test Loader
torch.manual_seed(8499)
seed(4899)
test_loader = torch.utils.data.DataLoader(test_data,
                                         batch_size=batch_size,
                                         shuffle = True)

In [35]:
class AttentionLayer(nn.Module):
    def __init__(self, in_dim1, in_dim2, attn_features, red_size, normalize_attn=True):
        super(AttentionLayer, self).__init__()
        self.red_size = red_size
        self.normalize_attn = normalize_attn
        self.W1 = nn.Conv2d(in_channels=in_dim1, out_channels=attn_features, kernel_size=1,padding=
0, bias=False)
        self.W2 = nn.Conv2d(in_channels=in_dim2, out_channels=attn_features, kernel_size=1,padding=
0, bias=False)
        self.W3 = nn.Conv2d(in_channels=attn_features, out_channels=1, kernel_size = 1, padding = 0, bias = True)
        
    def forward(self, l, g):
        N, C, W, H = l.size()
        l_ = self.W1(l)
        g_ = self.W2(g)
        
        if self.red_size > 1:
            g_ = F.interpolate(g_, size = self.red_size, mode = 'bilinear', align_corners=False)
        c = self.W3(torch.add(l_, g_))
        
        # Computa o mapa do attention
        if self.normalize_attn:
            a = F.softmax(c.view(N,1,-1), dim = 2).view(N,1,W,H)
        else:
            a = torch.sigmoid(c)
    
        # Re-weight the local feature
        f = torch.mul(a.expand_as(l), l) # batch_size CxWxH
    
        if self.normalize_attn:
            output = f.view(N,C,-1).sum(dim=2) # weighted sum
        else:
            output = F.adaptive_avg_pool2d(f, (1,1)).view(N,C) # global average pooling
        return output

In [36]:
# Arquitetura do Modelo
class ModeloCNN(nn.Module):
    
    # Pool1 = batch_size,16,50,50
    # Pool2 = batch_size,32,44,44
    # Pool3 = batch_size,64,20,20
    # Pool4 = batch_size,128,14,14
    
    # Método construtor
    def __init__(self):
        super(ModeloCNN, self).__init__()
        
        # Camada Convolucional de entrada 
        self.conv1 = nn.Conv2d(3, 16, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2, padding=1)
        
        # Camada de Dropout 1 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional 2 
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 1)
        
        # Camada MaxPooling 2
        self.maxpool = nn.MaxPool2d(kernel_size=5, stride=1, padding=0)
        
        # Camada de Dropout 2 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional 3 
        self.conv3 = nn.Conv2d(32, 64, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=1, padding=0)
        
        # Camada de Dropout 3 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional 4
        self.conv4 = nn.Conv2d(64, 128, kernel_size = 5, stride = 1, padding = 0)
        
        # Camada MaxPooling 4
        self.maxpool4 = nn.MaxPool2d(kernel_size = 5, stride = 1, padding = 1)
        
        # Camada de Attention 1
        self.Attn1 = AttentionLayer(in_dim1 = 32, in_dim2 = 128, attn_features = 256, normalize_attn = True, red_size = 44)
        
        # Camada de Attention 2
        self.Attn2 = AttentionLayer(in_dim1 = 64, in_dim2 = 128, attn_features = 256, normalize_attn = True, red_size = 20)
        
        # Camada Totalmente Conectada 1
        self.fc1 = nn.Linear(25184, 1000)
        
        # Camada Totalmente Conectada 2
        self.fc2 = nn.Linear(1000, 500)
        
        # Camada Totalmente Conectada 3
        self.fc3 = nn.Linear(500,4)
    
    # Método Forward
    def forward(self, x):
        
        # Adiciona uma camada de ativação Relu para cada camada convolucional
        pool1 = self.dropout(self.maxpool1(F.relu(self.conv1(x))))
        pool2 = self.dropout(self.maxpool(F.relu(self.conv2(pool1))))
        pool3 = self.dropout(self.maxpool3(F.relu(self.conv3(pool2))))
        pool4 = self.dropout(self.maxpool4(F.relu(self.conv4(pool3))))
        
        N, _, _, _ = pool4.size()
        
        # Faz o "achatamento" da matriz resultante da convolução e cria um vetor
        g = pool4.view(pool4.size(0), 14*14*128)
        
        # Camada de Attention 1
        g1 = self.Attn1(pool2, pool4)
        
        # Camada de Attention 2
        g2 = self.Attn2(pool3, pool4)
        
        # Concatenação dos resultados
        g_hat = torch.cat((g, g1, g2), dim = 1) # batch_size x C
        
        # Adiciona uma camada de dropout para regularização
        g_hat = self.dropout(g_hat)        
        
        # Camada Totalmente Conectada 1
        out = self.fc1(g_hat)
        
        # Camada Totalmente Conectada 2
        out = self.fc2(out)
        
        # Camada Totalmente Conectada 3 (Previsões do modelo)
        out = self.fc3(out)        
        
        return out

In [37]:
# Cria o modelo
modelo = ModeloCNN()
print(modelo)

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=4, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=5, stride=1, padding=1, dilation=1, ceil_mode=False)
  (Attn1): AttentionLayer(
    (W1): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (W2): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (W3): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
  )
  (Attn2): AttentionLayer(
    (W1): Conv2d(64, 256, kernel_s

In [38]:
# Movemos o modelo para a GPU se disponível
if torch.cuda.is_available():
    modelo.cuda()

In [39]:
# Função perda como categorical cross-entrupy
criterion = nn.CrossEntropyLoss()

In [48]:
# Hiperparâmetro taxa de aprendizado
lr = 0.00001

In [49]:
# Otimizador
optimizer = optim.Adam(modelo.parameters(), lr = lr)

## Treinamento

In [50]:
# Número de épocas para treinar o modelo
num_epochs = 300

In [51]:
# Hiperparametro para controlar a mudança do erro em validação
erro_valid_min = np.Inf

In [52]:
torch.cuda.is_available()

True

### Treinamento do modelo (a execução pode demorar)

In [53]:
%%time

torch.manual_seed(8499)
seed(8499)

for epoch in range(1, num_epochs + 1):

    # Parâmetros para acompanhar o erro total em treinamento e validação
    erro_treino = 0.0
    erro_valid = 0.0
    
    # Inicia o treinamento do modelo
    modelo.train().cuda()
    
    # Loop pelos batches de dados de treino
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # Move os tensores para a GPU se disponível
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        
        # Limpa os gradientes de todas as variáveis otimizadas
        optimizer.zero_grad()
        
        # Forward: calcula as saídas previstas
        output = modelo(data)
        
        # Calcula o erro no batch
        loss = criterion(output, target)
        
        # Backward: calcula o gradiente da perda em relação aos parâmetros do modelo
        loss.backward()
        
        # Realiza uma única etapa de otimização (atualização dos parâmetros)
        optimizer.step()
        
        # Atualiza o erro total em treino
        erro_treino += loss.item() * data.size(0)
        
    # Inicia a validação do modelo
    modelo.eval().cuda()
    
    # Loop pelos batches de dados de validação
    for batch_idx, (data, target) in enumerate(valid_loader):
        
        # Move os tensores para a GPU se disponível
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        
        # Forward: calcula as saídas previstas
        output = modelo(data)
        
        # Calcula o erro no batch
        loss = criterion(output, target)
        
        # Atualiza o erro total de validação
        erro_valid += loss.item() * data.size(0)
    
    # Calcula o erro médio
    erro_treino = erro_treino / len(train_loader.dataset)
    erro_valid = erro_valid / len(valid_loader.dataset)
        
    # Print
    print('\nEpoch: {} \tErro em Treinamento: {:.6f} \tErro em Validação: {:.6f}'.format(epoch, 
                                                                                         erro_treino, 
                                                                                         erro_valid))
    
    # Salva o modelo sempre que a perda em validação diminuir
    if erro_valid <= erro_valid_min:
        print('Erro em Validação foi Reduzido ({:.6f} --> {:.6f}). Salvando o modelo...'.format(erro_valid_min,
                                                                                                 erro_valid))
        pickle.dump(modelo, open('modelo_torch_attn2.pkl', 'wb'))
        erro_valid_min = erro_valid


Epoch: 1 	Erro em Treinamento: 0.005978 	Erro em Validação: 0.043767
Erro em Validação foi Reduzido (inf --> 0.043767). Salvando o modelo...

Epoch: 2 	Erro em Treinamento: 0.002348 	Erro em Validação: 0.043917

Epoch: 3 	Erro em Treinamento: 0.003434 	Erro em Validação: 0.036709
Erro em Validação foi Reduzido (0.043767 --> 0.036709). Salvando o modelo...

Epoch: 4 	Erro em Treinamento: 0.001634 	Erro em Validação: 0.039104

Epoch: 5 	Erro em Treinamento: 0.003019 	Erro em Validação: 0.038704

Epoch: 6 	Erro em Treinamento: 0.001682 	Erro em Validação: 0.038084

Epoch: 7 	Erro em Treinamento: 0.001531 	Erro em Validação: 0.036587
Erro em Validação foi Reduzido (0.036709 --> 0.036587). Salvando o modelo...

Epoch: 8 	Erro em Treinamento: 0.001190 	Erro em Validação: 0.036736

Epoch: 9 	Erro em Treinamento: 0.001844 	Erro em Validação: 0.033888
Erro em Validação foi Reduzido (0.036587 --> 0.033888). Salvando o modelo...

Epoch: 10 	Erro em Treinamento: 0.001975 	Erro em Validação: 0.036


Epoch: 110 	Erro em Treinamento: 0.000585 	Erro em Validação: 0.037920

Epoch: 111 	Erro em Treinamento: 0.000465 	Erro em Validação: 0.038169

Epoch: 112 	Erro em Treinamento: 0.001166 	Erro em Validação: 0.037726

Epoch: 113 	Erro em Treinamento: 0.000376 	Erro em Validação: 0.045191

Epoch: 114 	Erro em Treinamento: 0.000235 	Erro em Validação: 0.037414

Epoch: 115 	Erro em Treinamento: 0.001074 	Erro em Validação: 0.035387

Epoch: 116 	Erro em Treinamento: 0.000725 	Erro em Validação: 0.045292

Epoch: 117 	Erro em Treinamento: 0.000650 	Erro em Validação: 0.050120

Epoch: 118 	Erro em Treinamento: 0.000456 	Erro em Validação: 0.043178

Epoch: 119 	Erro em Treinamento: 0.000464 	Erro em Validação: 0.037081

Epoch: 120 	Erro em Treinamento: 0.000953 	Erro em Validação: 0.040652

Epoch: 121 	Erro em Treinamento: 0.000943 	Erro em Validação: 0.040222

Epoch: 122 	Erro em Treinamento: 0.000662 	Erro em Validação: 0.039834

Epoch: 123 	Erro em Treinamento: 0.000293 	Erro em Validação: 0


Epoch: 224 	Erro em Treinamento: 0.000113 	Erro em Validação: 0.040241

Epoch: 225 	Erro em Treinamento: 0.000241 	Erro em Validação: 0.044708

Epoch: 226 	Erro em Treinamento: 0.000028 	Erro em Validação: 0.045934

Epoch: 227 	Erro em Treinamento: 0.000397 	Erro em Validação: 0.042084

Epoch: 228 	Erro em Treinamento: 0.000984 	Erro em Validação: 0.045188

Epoch: 229 	Erro em Treinamento: 0.001141 	Erro em Validação: 0.047559

Epoch: 230 	Erro em Treinamento: 0.000033 	Erro em Validação: 0.043430

Epoch: 231 	Erro em Treinamento: 0.000742 	Erro em Validação: 0.044825

Epoch: 232 	Erro em Treinamento: 0.000291 	Erro em Validação: 0.044045

Epoch: 233 	Erro em Treinamento: 0.000074 	Erro em Validação: 0.047885

Epoch: 234 	Erro em Treinamento: 0.000250 	Erro em Validação: 0.042088

Epoch: 235 	Erro em Treinamento: 0.001186 	Erro em Validação: 0.046956

Epoch: 236 	Erro em Treinamento: 0.000315 	Erro em Validação: 0.039282

Epoch: 237 	Erro em Treinamento: 0.000249 	Erro em Validação: 0

In [54]:
# Carregando o modelo com menor erro de validação após treinamanto
modelo = pickle.load(open('modelo_torch_attn2.pkl', 'rb'))

In [44]:
# Só usar caso a acurácia do modelo testado seja maior que a acurácia atual do arquivo modelo_torch.pkl
#pickle.dump(modelo, open('modelo_torch_attn.pkl', 'wb'))

### Testando e Avaliando o Modelo Final

In [45]:
# Lista de classes das imagens
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [46]:
# Erro em teste
erro_teste = 0.0

In [47]:
# Controle de acertos do modelo
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

In [56]:
%%time

torch.manual_seed(8499)
seed(4899)

prediction = torch.Tensor()
prediction = prediction.cuda()
y_test = torch.Tensor()
y_test = y_test.cuda()

torch.manual_seed(8499)
seed(8499)

# Lista de classes das imagens
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Erro em teste
erro_teste = 0.0

# Controle de acertos do modelo
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

# Inicia a avaliação do modelo
modelo.eval()

# Loop pelos batches de dados de teste
for batch_idx, (data, target) in enumerate(test_loader):
    
    # Move os tensores para GPU se disponível
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
        y_test = torch.cat((y_test, target), 0)
    
    # Forward
    output = modelo(data)
    
    # Calcula o erro
    loss = criterion(output, target)
    
    # Atualiza o erro em teste
    erro_teste += loss.item() * data.size(0)
    
    # Converte probabilidades de saída em classe prevista
    _, pred = torch.max(output, 1)
    prediction = torch.cat((prediction, pred), 0)
    
    # Compara as previsões com o rótulo verdadeiro
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    
    # Calcula a precisão do teste para cada classe
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# Erro médio em teste
erro_teste = erro_teste / len(test_loader.dataset)
print('\nErro em Teste: {:.6f}\n'.format(erro_teste))

# Calcula a acurácia para cada classe
for i in range(4):
    if class_total[i] > 0:
        print('Acurácia em Teste da classe %5s: %2d%% (%2d/%2d)' % (classes[i], 
                                                             100 * class_correct[i] / class_total[i],
                                                             np.sum(class_correct[i]), 
                                                             np.sum(class_total[i])))
    else:
        print('Acurácia em Teste de %5s:)' % (classes[i]))

# Calcula a acurácia total
print('\nAcurácia em Teste (Total): %.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                        np.sum(class_correct), 
                                                        np.sum(class_total)))

y_test = y_test.cpu()
prediction = prediction.cpu()
cf_matrix = confusion_matrix(y_test, prediction)
print("\nMatriz de Confusão do Modelo com Camadas de Attention:")
print(cf_matrix)
print("\n")


Erro em Teste: 0.072035

Acurácia em Teste da classe glioma: 98% (294/300)
Acurácia em Teste da classe meningioma: 96% (294/306)
Acurácia em Teste da classe notumor: 100% (405/405)
Acurácia em Teste da classe pituitary: 99% (298/300)

Acurácia em Teste (Total): 98.47% (1291/1311)

Matriz de Confusão do Modelo com Camadas de Attention:
[[294   5   0   1]
 [  2 294   5   5]
 [  0   0 405   0]
 [  1   1   0 298]]


CPU times: user 22.9 s, sys: 138 ms, total: 23 s
Wall time: 3.6 s


## Teste do modelo atual (modelo_torch_attn) {modelo com a melhor acurácia atual}

In [27]:
class AttentionLayer(nn.Module):
    def __init__(self, in_dim1, in_dim2, attn_features, red_size, normalize_attn=True):
        super(AttentionLayer, self).__init__()
        self.red_size = red_size
        self.normalize_attn = normalize_attn
        self.W1 = nn.Conv2d(in_channels=in_dim1, out_channels=attn_features, kernel_size=1,padding=
0, bias=False)
        self.W2 = nn.Conv2d(in_channels=in_dim2, out_channels=attn_features, kernel_size=1,padding=
0, bias=False)
        self.W3 = nn.Conv2d(in_channels=attn_features, out_channels=1, kernel_size = 1, padding = 0, bias = True)
        
    def forward(self, l, g):
        N, C, W, H = l.size()
        l_ = self.W1(l)
        g_ = self.W2(g)
        
        if self.red_size > 1:
            g_ = F.interpolate(g_, size = self.red_size, mode = 'bilinear', align_corners=False)
        c = self.W3(torch.add(l_, g_))
        
        # Computa o mapa do attention
        if self.normalize_attn:
            a = F.softmax(c.view(N,1,-1), dim = 2).view(N,1,W,H)
        else:
            a = torch.sigmoid(c)
    
        # Re-weight the local feature
        f = torch.mul(a.expand_as(l), l) # batch_size CxWxH
    
        if self.normalize_attn:
            output = f.view(N,C,-1).sum(dim=2) # weighted sum
        else:
            output = F.adaptive_avg_pool2d(f, (1,1)).view(N,C) # global average pooling
        return output

In [28]:
# Arquitetura do Modelo
class ModeloCNN(nn.Module):
    
    # Pool1 = batch_size,16,50,50
    # Pool2 = batch_size,32,44,44
    # Pool3 = batch_size,64,20,20
    # Pool4 = batch_size,128,14,14
    
    # Método construtor
    def __init__(self):
        super(ModeloCNN, self).__init__()
        
        # Camada Convolucional de entrada 
        self.conv1 = nn.Conv2d(3, 16, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2, padding=1)
        
        # Camada de Dropout 1 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional 2 
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 1)
        
        # Camada MaxPooling 2
        self.maxpool = nn.MaxPool2d(kernel_size=5, stride=1, padding=0)
        
        # Camada de Dropout 2 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional 3 
        self.conv3 = nn.Conv2d(32, 64, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=1, padding=0)
        
        # Camada de Dropout 3 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional 4
        self.conv4 = nn.Conv2d(64, 128, kernel_size = 5, stride = 1, padding = 0)
        
        # Camada MaxPooling 4
        self.maxpool4 = nn.MaxPool2d(kernel_size = 5, stride = 1, padding = 1)
        
        # Camada de Attention 1
        self.Attn1 = AttentionLayer(in_dim1 = 32, in_dim2 = 128, attn_features = 256, normalize_attn = True, red_size = 44)
        
        # Camada de Attention 2
        self.Attn2 = AttentionLayer(in_dim1 = 64, in_dim2 = 128, attn_features = 256, normalize_attn = True, red_size = 20)
        
        # Camada Totalmente Conectada 1
        self.fc1 = nn.Linear(25184, 1000)
        
        # Camada Totalmente Conectada 2
        self.fc2 = nn.Linear(1000, 500)
        
        # Camada Totalmente Conectada 3
        self.fc3 = nn.Linear(500,4)
    
    # Método Forward
    def forward(self, x):
        
        # Adiciona uma camada de ativação Relu para cada camada convolucional
        pool1 = self.dropout(self.maxpool1(F.relu(self.conv1(x))))
        pool2 = self.dropout(self.maxpool(F.relu(self.conv2(pool1))))
        pool3 = self.dropout(self.maxpool3(F.relu(self.conv3(pool2))))
        pool4 = self.dropout(self.maxpool4(F.relu(self.conv4(pool3))))
        
        N, _, _, _ = pool4.size()
        
        # Faz o "achatamento" da matriz resultante da convolução e cria um vetor
        g = pool4.view(x.size(0), 14*14*128)
        
        # Camada de Attention 1
        g1 = self.Attn1(pool2, pool4)
        
        # Camada de Attention 2
        g2 = self.Attn2(pool3, pool4)
        
        # Concatenação dos resultados
        g_hat = torch.cat((g, g1, g2), dim = 1) # batch_size x C
        
        # Adiciona uma camada de dropout para regularização
        g_hat = self.dropout(g_hat)        
        
        # Camada Totalmente Conectada 1
        out = self.fc1(g_hat)
        
        # Camada Totalmente Conectada 2
        out = self.fc2(out)
        
        # Camada Totalmente Conectada 3 (Previsões do modelo)
        out = self.fc3(out)        
        
        return out

In [60]:
# Carregando o modelo com menor erro de validação geral, em arquivo salvo modelo_torch.pkl
modelo_atual = pickle.load(open('modelo_torch_attn2.pkl', 'rb'))
print(modelo_atual)

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=4, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=5, stride=1, padding=1, dilation=1, ceil_mode=False)
  (Attn1): AttentionLayer(
    (W1): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (W2): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (W3): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
  )
  (Attn2): AttentionLayer(
    (W1): Conv2d(64, 256, kernel_s

In [63]:
%%time

torch.manual_seed(8499)
seed(4899)

prediction = torch.Tensor()
prediction = prediction.cuda()
y_test = torch.Tensor()
y_test = y_test.cuda()

torch.manual_seed(8499)
seed(8499)

# Lista de classes das imagens
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Erro em teste
erro_teste = 0.0

# Controle de acertos do modelo
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

# Inicia a avaliação do modelo
modelo.eval()

# Loop pelos batches de dados de teste
for batch_idx, (data, target) in enumerate(test_loader):
    
    # Move os tensores para GPU se disponível
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
        y_test = torch.cat((y_test, target), 0)
    
    # Forward
    output = modelo(data)
    
    # Calcula o erro
    loss = criterion(output, target)
    
    # Atualiza o erro em teste
    erro_teste += loss.item() * data.size(0)
    
    # Converte probabilidades de saída em classe prevista
    _, pred = torch.max(output, 1)
    prediction = torch.cat((prediction, pred), 0)
    
    # Compara as previsões com o rótulo verdadeiro
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    
    # Calcula a precisão do teste para cada classe
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# Erro médio em teste
erro_teste = erro_teste / len(test_loader.dataset)
print('\nErro em Teste: {:.6f}\n'.format(erro_teste))

# Calcula a acurácia para cada classe
for i in range(4):
    if class_total[i] > 0:
        print('Acurácia em Teste da classe %5s: %2d%% (%2d/%2d)' % (classes[i], 
                                                             100 * class_correct[i] / class_total[i],
                                                             np.sum(class_correct[i]), 
                                                             np.sum(class_total[i])))
    else:
        print('Acurácia em Teste de %5s:)' % (classes[i]))

# Calcula a acurácia total
print('\nAcurácia em Teste (Total): %.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                        np.sum(class_correct), 
                                                        np.sum(class_total)))

y_test = y_test.cpu()
prediction = prediction.cpu()
cf_matrix = confusion_matrix(y_test, prediction)
print("\nMatriz de Confusão do Modelo com Camadas de Attention:")
print(cf_matrix)
print("\n")


Erro em Teste: 0.096220

Acurácia em Teste da classe glioma: 96% (290/300)
Acurácia em Teste da classe meningioma: 96% (296/306)
Acurácia em Teste da classe notumor: 99% (402/405)
Acurácia em Teste da classe pituitary: 99% (299/300)

Acurácia em Teste (Total): 98.17% (1287/1311)

Matriz de Confusão do Modelo com Camadas de Attention:
[[290   9   0   1]
 [  4 296   2   4]
 [  0   2 402   1]
 [  0   1   0 299]]


CPU times: user 22.6 s, sys: 69.3 ms, total: 22.7 s
Wall time: 4.17 s


In [89]:
print("Métricas de performance do modelo com camada de attention:")
print("Accuracy: ", accuracy_score(y_test, prediction))
print("Precision: ",precision_score(y_test, prediction, average='macro'))
print("F1-Score: ",f1_score(y_test, prediction, average='macro'))
print("Recall: ",recall_score(y_test, prediction, average='macro'))

Métricas de performance do modelo com camada de attention:
Accuracy:  0.9816933638443935
Precision:  0.9807027231662611
F1-Score:  0.9807124099722214
Recall:  0.9808115468409586
